In [1]:
%pip install pandas pysus

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 11.1 MB/s  0:00:011.2 MB/s eta 0:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 11.4 MB/s  0:00:01 11.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 MB 10.4 MB/s  0:00:04 10.5 MB/s eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 kB 8.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 7.7 MB/s  0:00:00
Using cache

In [7]:
import os
import ftplib
import pandas as pd
import pyreaddbc
from dbfread import DBF
import numpy as np


In [3]:
def main():
    try:
        script_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        script_dir = os.getcwd()

    raw_dir = os.path.join(script_dir, "dados")
    processed_dir = os.path.join(script_dir, "dados_processados")

    os.makedirs(raw_dir, exist_ok=True)
    os.makedirs(processed_dir, exist_ok=True)

    ftp_host = "ftp.datasus.gov.br"
    ftp_path = "/dissemin/publicos/SIM/CID10/DORES/"
    filename = "DOSP2023.dbc"

    local_dbc = os.path.join(raw_dir, filename)
    local_dbf = os.path.join(raw_dir, filename.replace('.dbc', '.dbf'))
    local_csv = os.path.join(processed_dir, filename.replace('.dbc', '.csv'))

    print(f"Iniciando conexão com {ftp_host}...")

    try:
        ftp = ftplib.FTP(ftp_host)
        ftp.login()
        ftp.cwd(ftp_path)

        print(f"Baixando {filename}...")

        with open(local_dbc, 'wb') as f:
            ftp.retrbinary(f"RETR {filename}", f.write)

        ftp.quit()
        print("Download concluído.")

    except ftplib.all_errors as e:
        print(f"Erro no FTP: {e}")
        return

    print("Iniciando conversão para CSV...")

    try:
        pyreaddbc.dbc2dbf(local_dbc, local_dbf)

        table = DBF(local_dbf, encoding='iso-8859-1')
        df = pd.DataFrame(iter(table))

        df.to_csv(local_csv, index=False, encoding='utf-8')

        if os.path.exists(local_dbf):
            os.remove(local_dbf)

        print(f"Sucesso! Arquivo salvo em: {local_csv}")
        print(f"Total de registros processados: {len(df)}")

    except Exception as e:
        print(f"Erro na conversão: {e}")

if __name__ == "__main__":
    main()

Iniciando conexão com ftp.datasus.gov.br...
Baixando DOSP2023.dbc...
Download concluído.
Iniciando conversão para CSV...
Sucesso! Arquivo salvo em: /home/carolina/Documents/TCC/TCC/notebooks/dados_processados/DOSP2023.csv
Total de registros processados: 334303


In [10]:
file_path = "/home/carolina/Documents/TCC/TCC/notebooks/dados_processados/DOSP2023.csv"
df = pd.read_csv(file_path)

print(f"DataFrame loaded from: {file_path}")
print(f"Total records: {len(df)}")
df.head()

DataFrame loaded from: /home/carolina/Documents/TCC/TCC/notebooks/dados_processados/DOSP2023.csv
Total records: 334303


/tmp/ipykernel_255793/1404854105.py:2: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,FONTES,TPRESGINFO,TPNIVELINV,NUDIASINF,DTCADINF,MORTEPARTO,DTCONCASO,FONTESINF,ALTCAUSA,CONTADOR
0,1,2,1012023,720.0,835.0,351492.0,8011945.0,477,2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,1,2,1012023,2300.0,822.0,220010.0,2091979.0,443,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,1,2,1012023,2150.0,835.0,350400.0,17122022.0,215,1,1.0,...,SXXXXX,NaN,NaN,NaN,2022023.0,3.0,2022023.0,NaN,2.0,5
3,1,2,1072023,815.0,841.0,410050.0,7121989.0,433,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
4,1,2,1072023,1935.0,825.0,251660.0,6021942.0,481,2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7


In [15]:
def count_missing_values(column):
    null_count = df[column].isnull().sum()
    return null_count


In [19]:
all_columns= df.columns
missing_values = {col: count_missing_values(col) for col in all_columns}
print("Missing values per column:")


# for col, count in missing_values.items():
#     print(f"{col}: {count}")   


missing_values.TPOBITOCOR

Missing values per column:


AttributeError: 'dict' object has no attribute 'TPOBITOCOR'

In [25]:
# Exemplo para limpar e agrupar
df['CID_LIMPO'] = df['LINHAA'].str.replace('*', '', regex=False)
df['GRUPO_CID'] = df['CID_LIMPO'].str[:3] # Pega os 3 primeiros caracteres (ex: I21)

1. Limpeza e Padronização
Os dados brutos do SIH vêm com "sujeiras" de sistema que dificultam a análise.
Remover caracteres especiais: Como você notou, os códigos vêm com um asterisco (*). O primeiro passo é removê-lo para que o código fique apenas como I119.
Tratar valores vazios (NaN): Muitas linhas podem não ter diagnóstico secundário. Você precisa decidir se vai excluir essas linhas ou preenchê-las com algo como "Não informado" para não dar erro nos cálculos.
Ajustar o tamanho do código: Às vezes o CID tem 3 caracteres, às vezes 4. Padronize todos para o mesmo formato para conseguir cruzar com a tabela de nomes.
2. Tradução (O "De-Para")
Ter apenas I219 não ajuda muito na leitura do relatório.
Você deve pegar uma tabela de referência (o dicionário da CID-10) e dizer ao computador: "Sempre que encontrar I219, escreva ao lado 'Infarto agudo do miocárdio'". Isso torna seus gráficos e tabelas compreensíveis para qualquer pessoa.
3. Agrupamento por Categorias
Analisar código por código (ex: I210, I211, I212) gera tabelas gigantescas e confusas.
Criar Grupos: O ideal é agrupar. Em vez de 50 tipos de infarto, você cria uma categoria única chamada "Doenças Isquêmicas".
Isso ajuda a identificar padrões maiores, como: "As doenças isquêmicas representam 40% das internações cardíacas".
4. Filtragem de Interesse
Os arquivos do SIH costumam ser enormes (milhares de linhas).
Nesta etapa, você descarta tudo o que não é cardíaco. Você filtra o banco de dados para manter apenas as linhas onde o código começa com a letra "I" (ou os intervalos específicos que vimos antes). Isso deixa o processamento muito mais rápido.
5. Cruzamento com Desfechos (O "Pulo do Gato")
O CID sozinho diz o que a pessoa tem, mas você precisa saber o que aconteceu:
Mortalidade: Verifique a coluna de "Motivo da Saída" ou "Óbito". Se o motivo for óbito, você marca aquela linha.
Tempo de permanência: Calcule a diferença entre a data de internação e a data de saída para ver quais doenças cardíacas seguram o paciente por mais tempo no hospital.
Custos: Veja o valor total pago pela AIH para entender o impacto financeiro de cada doença.
6. Verificação de Consistência
Antes de finalizar, faça uma "checagem de realidade":
Existem idades impossíveis (ex: 150 anos)?
Existem diagnósticos de gravidez em homens?
Existem diagnósticos cardíacos infantis em alas geriátricas?

merge com CID10MORB.csv


In [24]:
df_cardio = df[df['LINHAA'].str.contains(r'^\*I(0[0-9]|[1-4][0-9]|5[0-2])', na=False)]

df_cardio['LINHAA']


/tmp/ipykernel_255793/2187270669.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_cardio = df[df['LINHAA'].str.contains(r'^\*I(0[0-9]|[1-4][0-9]|5[0-2])', na=False)]


3         *I119
6         *I219
26        *I461
28        *I461
31        *I269
          ...  
334267    *I469
334280    *I499
334282    *I219
334284    *I509
334288    *I461
Name: LINHAA, Length: 42313, dtype: object

### Descrição das Colunas do DataFrame

Aqui está uma interpretação do significado de cada coluna no seu DataFrame `df`:

*   **`ORIGEM`**: Indica a origem do registro (e.g., tipo de declaração de óbito).
*   **`TIPOBITO`**: Tipo de óbito (e.g., fetal, não fetal).
*   **`DTOBITO`**: Data do óbito.
*   **`HORAOBITO`**: Hora do óbito.
*   **`NATURAL`**: Naturalidade (código do município ou estado de nascimento).
*   **`CODMUNNATU`**: Código do município de nascimento.
*   **`DTNASC`**: Data de nascimento do falecido.
*   **`IDADE`**: Idade do falecido (frequentemente codificada, e.g., em dias, meses, anos).
*   **`SEXO`**: Sexo do falecido (e.g., 1 para masculino, 2 para feminino).
*   **`RACACOR`**: Raça/Cor do falecido (e.g., branca, preta, parda, amarela, indígena).
*   **`ESTCIV`**: Estado civil do falecido.
*   **`ESC`**: Escolaridade do falecido (nível de instrução).
*   **`ESC2010`**: Escolaridade do falecido conforme a classificação de 2010.
*   **`SERIESCFAL`**: Série escolar do falecido (nível de ensino).
*   **`OCUP`**: Ocupação do falecido.
*   **`CODMUNRES`**: Código do município de residência do falecido.
*   **`LOCOCOR`**: Local de ocorrência do óbito (e.g., hospital, domicílio, via pública).
*   **`CODESTAB`**: Código do estabelecimento de saúde onde ocorreu o óbito (se aplicável).
*   **`ESTABDESCR`**: Descrição do estabelecimento (Nome do Hospital, por exemplo).
*   **`CODMUNOCOR`**: Código do município de ocorrência do óbito.
*   **`IDADEMAE`**: Idade da mãe (para óbitos infantis ou maternos).
*   **`ESCMAE`**: Escolaridade da mãe.
*   **`ESCMAE2010`**: Escolaridade da mãe conforme a classificação de 2010.
*   **`SERIESCMAE`**: Série escolar da mãe.
*   **`OCUPMAE`**: Ocupação da mãe.
*   **`QTDFILVIVO`**: Quantidade de filhos vivos (para mães).
*   **`QTDFILMORT`**: Quantidade de filhos mortos (para mães).
*   **`GRAVIDEZ`**: Status de gravidez (relacionado a óbitos maternos).
*   **`SEMAGESTAC`**: Semanas de gestação (para óbitos fetais ou infantis).
*   **`GESTACAO`**: Tipo de gestação.
*   **`PARTO`**: Tipo de parto.
*   **`OBITOPARTO`**: Se o óbito ocorreu durante o parto (sim/não).
*   **`PESO`**: Peso ao nascer (para óbitos infantis).
*   **`TPMORTEOCO`**: Tipo de morte na ocorrência.
*   **`OBITOGRAV`**: Se o óbito foi relacionado à gravidez.
*   **`OBITOPUERP`**: Se o óbito foi relacionado ao puerpério (período pós-parto).
*   **`ASSISTMED`**: Se houve assistência médica no momento do óbito.
*   **`EXAME`**: Se foi realizado algum exame (e.g., laboratorial).
*   **`CIRURGIA`**: Se foi realizada alguma cirurgia (os valores 1, 2, 9 e NaN provavelmente significam 'Sim', 'Não', 'Ignorado' e 'Ausente', respectivamente).
*   **`NECROPSIA`**: Se foi realizada necropsia.
*   **`LINHAA`, `LINHAB`, `LINHAC`, `LINHAD`**: Causas de óbito listadas sequencialmente na Declaração de Óbito (parte I), representando a cadeia de eventos que levou à morte.
*   **`LINHAII`**: Outras condições significativas que contribuíram para a morte, mas não faziam parte da cadeia direta de eventos.
*   **`CAUSABAS`**: Causa básica do óbito (a doença ou lesão que iniciou a cadeia de eventos mórbidos).
*   **`CB_PRE`**: Causa básica pré-codificada.
*   **`COMUNSVOIM`**: Provavelmente alguma variável comum para o Sistema de Informações sobre Mortalidade (SIM).
*   **`DTATESTADO`**: Data de atestado do óbito.
*   **`CIRCOBITO`**: Circunstância do óbito (e.g., acidente, suicídio, homicídio).
*   **`ACIDTRAB`**: Se foi acidente de trabalho.
*   **`FONTE`**: Fonte da informação.
*   **`NUMEROLOTE`**: Número do lote ao qual o registro pertence.
*   **`TPPOS`**: Tipo de processamento ou posição.
*   **`DTINVESTIG`**: Data da investigação do óbito (se aplicável).
*   **`CAUSABAS_O`**: Causa básica original (antes de possíveis correções).
*   **`DTCADASTRO`**: Data de cadastro do registro.
*   **`ATESTANTE`**: Informações sobre o profissional que atestou o óbito.
*   **`STCODIFICA`**: Status da codificação.
*   **`CODIFICADO`**: Se o óbito foi codificado (sim/não).
*   **`VERSAOSIST`**: Versão do sistema utilizado para o registro.
*   **`VERSAOSCB`**: Versão da SCB (provavelmente da Classificação Brasileira de Ocupações ou similar).
*   **`FONTEINV`**: Fonte da investigação.
*   **`DTRECEBIM`**: Data de recebimento do registro.
*   **`ATESTADO`**: Se o óbito foi atestado.
*   **`DTRECORIGA`**: Data do registro original.
*   **`CAUSAMAT`**: Causa materna do óbito.
*   **`ESCMAEAGR1`**: Escolaridade da mãe agregada (nível mais amplo).
*   **`ESCFALAGR1`**: Escolaridade do falecido agregada.
*   **`STDOEPIDEM`**: Status de doença epidêmica.
*   **`STDONOVA`**: Status de doença nova.
*   **`DIFDATA`**: Diferença de datas (e.g., entre óbito e registro).
*   **`NUDIASOBCO`**: Número de dias de observação de co-ocorrências.
*   **`NUDIASOBIN`**: Número de dias de observação em investigação.
*   **`DTCADINV`**: Data de cadastro da investigação.
*   **`TPOBITOCOR`**: Tipo de ocorrência de óbito.
*   **`DTCONINV`**: Data de conclusão da investigação.
*   **`FONTES`**: Fontes adicionais.
*   **`TPRESGINFO`**: Tipo de registro de informação.
*   **`TPNIVELINV`**: Tipo de nível de investigação.
*   **`NUDIASINF`**: Número de dias de internação ou doença.
*   **`DTCADINF`**: Data de cadastro de informações.
*   **`MORTEPARTO`**: Morte relacionada ao parto.
*   **`DTCONCASO`**: Data de conclusão do caso.
*   **`FONTESINF`**: Fontes de informação.
*   **`ALTCAUSA`**: Causa alternativa.
*   **`CONTADOR`**: Um contador ou identificador de registro.